# Exact QBM $s^*$ Comparison

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext autotime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import neptune.new as neptune
from scipy.constants import k as k_B, h as h_P

k_B /= h_P * 1e9

from qbm.utils import (
    binarize_df,
    convert_bin_list_to_str,
    get_binarization_params,
    get_project_dir,
    get_rng,
    kl_divergence,
    load_artifact,
    lr_exp_decay,
    prepare_training_data,
    save_artifact,
    unbinarize_df,
)
from qbm.models import BQRBM

# configure directories
project_dir = get_project_dir()
artifact_dir = project_dir / "artifacts/qbm/7x3"
if not artifact_dir.exists():
    artifact_dir.mkdir(parents=True)
plot_dir = project_dir / "results/plots/qbm/7x3"
if not plot_dir.exists():
    plot_dir.mkdir(parents=True)
    
# load anneal schedule
df_anneal = pd.read_csv(
    project_dir
    / "data/anneal_schedules/csv/09-1265A-A_Advantage_system5_1_annealing_schedule.csv",
    index_col="s",
)
if 0.5 not in df_anneal.index:
    df_anneal.loc[0.5] = (df_anneal.loc[0.499] + df_anneal.loc[0.501]) / 2

time: 1.89 s (started: 2022-02-24 09:43:06 +01:00)


## Train Data Creation

In [2]:
seed = 42
n_visible = 7
n_hidden = 3
n_qubits = n_visible + n_hidden

rng = get_rng(seed)
n_samples = 1500
α = 2 / 3
N_1 = rng.normal(-2, 1, int(round(n_samples * α, 0)))
N_2 = rng.normal(3, 1, int(round(n_samples * (1 - α), 0)))
x = np.concatenate((N_1, N_2))

df = pd.DataFrame.from_dict({"x": x})
binarization_params = get_binarization_params(df, n_bits=n_visible)
df_binarized = binarize_df(df, binarization_params)
X_train = prepare_training_data(df_binarized)["X_train"]

time: 8.06 ms (started: 2022-02-24 09:43:08 +01:00)


In [3]:
def callback(model, sample_state_vectors):
    X_train = model.X_train
    n_visible = model.X_train.shape[1]
    train_states = (
        model._eigen_to_binary(model.X_train) * 2.0 ** np.arange(n_visible - 1, -1, -1)
    ).sum(axis=1)
    sample_states = (
        model._eigen_to_binary(sample_state_vectors[:, :n_visible])
        * 2.0 ** np.arange(n_visible - 1, -1, -1)
    ).sum(axis=1)

    dkl = kl_divergence(train_states, sample_states, n_bins=32)

    return {"value": dkl, "print": f"D_KL = {dkl:.3f}"}

time: 71.2 ms (started: 2022-02-24 09:43:08 +01:00)


## Model Analysis (Exact)

In [21]:
# model params
embedding = None

# training params
n_epochs = 100
n_samples = 10_000
learning_rate = 0.1
mini_batch_size = 10
decay_epoch = 50
decay_period = 10
epochs = np.arange(1, n_epochs + 1)
learning_rates = learning_rate * lr_exp_decay(epochs, decay_epoch, decay_period)
learning_rates_beta = learning_rates
learning_rates_beta[:10] *= 5
learning_rates_beta[10:20] *= 2

train = True
s_values = np.round(np.arange(45, 105, 5) / 100, 2)
B_values = np.array([df_anneal.loc[s, "B(s) (GHz)"] for s in s_values])
beta_B_const = 80 / k_B
betas = beta_B_const / B_values
for i, s in enumerate(s_values):
    exact_params = {"beta": betas[i]}
    print(f"s = {s:.2f}, β = {betas[i]}")
    model_path = artifact_dir / f"models/model_exact-s={s:.2f}.pkl"
    if train:
        anneal_params = {
            "s": s,
            "A": df_anneal.loc[s, "A(s) (GHz)"],
            "B": df_anneal.loc[s, "B(s) (GHz)"],
        }
        # model init
        model = BQRBM(
            X_train=X_train,
            n_hidden=n_hidden,
            embedding=embedding,
            anneal_params=anneal_params,
            beta_initial=betas[i],
            beta_range=(1e-3, 10),
            exact_params=exact_params,
            seed=i,
        )

        # model train and save
        model.train(
            n_epochs=n_epochs,
            n_samples=n_samples,
            learning_rate=learning_rates,
            learning_rate_beta=0.0,
            mini_batch_size=mini_batch_size,
            callback=callback,
        )
        model.save(model_path)
    else:
        model = BQRBM.load(model_path)

s = 0.45, β = 1.4998191177770177
[BQRBM] epoch 1: β = 1.500, learning rate = 5.00e-01, β learning rate = 0.00e+00, epoch duration = 0:00:23.407855
D_KL = 0.077
[BQRBM] epoch 2: β = 1.500, learning rate = 5.00e-01, β learning rate = 0.00e+00, epoch duration = 0:00:23.451827
D_KL = 0.064
[BQRBM] epoch 3: β = 1.500, learning rate = 5.00e-01, β learning rate = 0.00e+00, epoch duration = 0:00:23.192144
D_KL = 0.057
[BQRBM] epoch 4: β = 1.500, learning rate = 5.00e-01, β learning rate = 0.00e+00, epoch duration = 0:00:22.973227
D_KL = 0.090
[BQRBM] epoch 5: β = 1.500, learning rate = 5.00e-01, β learning rate = 0.00e+00, epoch duration = 0:00:22.702656
D_KL = 0.073
[BQRBM] epoch 6: β = 1.500, learning rate = 5.00e-01, β learning rate = 0.00e+00, epoch duration = 0:00:22.685196
D_KL = 0.041
[BQRBM] epoch 7: β = 1.500, learning rate = 5.00e-01, β learning rate = 0.00e+00, epoch duration = 0:00:22.544190
D_KL = 0.032
[BQRBM] epoch 8: β = 1.500, learning rate = 5.00e-01, β learning rate = 0.00e+

PicklingError: Can't pickle <class 'qbm.models.BQRBM.BQRBM'>: it's not the same object as qbm.models.BQRBM.BQRBM

time: 37min 32s (started: 2022-02-24 12:21:41 +01:00)


In [5]:
model_paths = sorted(
    [
        f
        for f in (artifact_dir / "models").iterdir()
        if f.name.startswith("model_exact-s=")
    ]
)
models = [BQRBM.load(model_path) for model_path in model_paths]

time: 13.3 ms (started: 2022-01-31 17:26:45 +01:00)


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6), dpi=300)
import seaborn as sns

def smooth(x, k=10):
    x_out = np.zeros(len(x))
    for i in range(len(x)):
        x_out[i] = np.mean(x[i - min((k - 1, i)):i+1])
    return x_out

markers = ["o", "^", "v", "<", ">", "s", "p", "*", "P", "X", "+", "x"]
colors = [
    "tab:blue",
    "tab:orange",
    "tab:green",
    "tab:red",
    "tab:purple",
    "tab:brown",
    "tab:pink",
    "tab:gray",
    "tab:olive",
    "tab:cyan",
    "black",
    "gray",
]
# colors = sns.color_palette("magma", len(models))
s_betas = {s: beta for s, beta in zip(s_values, betas)}
for i, m in enumerate(models):
    dkls = [d["value"] for d in m.callback_outputs]
    print(m.anneal_params["s"], np.mean(dkls[-10:]))
    epochs = np.arange(1, len(dkls) + 1)
    ax.plot(
        epochs,
        smooth(dkls),
        label=fr"$s^* = {m.anneal_params['s']:.2f}$, $\beta_{{eff}} = {betas[::-1][i]:.2f}$",
        color=colors[i],
        marker=markers[i],
        markersize=8,
        markevery=3,
    )
ax.set_yticks(np.arange(0, 0.13, 0.02))
ax.set_ylim((0, 0.12))
ax.set_xlabel("Epoch")
ax.set_ylabel(r"$D_{KL}(p_{data} \ || \ p_{model})$ [10 Epoch Moving Average]")
ax.grid(True)
ax.legend()
plt.tight_layout()
plt.savefig(plot_dir / "dkl_exact_s_sweep.png")